In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import shapiro
import os
import seaborn as sns
from scipy.stats import norm


# Correlation matrix
With the .corr() the correlation matrix is displayed.

In [6]:
cor=df.corr()
cor

,AgriShareGDP,CreditToAgriFishForest,EmploymentRural,GDP,TotalAgri,% Soldiers,Birth Rate,Death Rate,Homicides,Life Expectancy,...,% Population Growth,% Rural Population,Civil Liberties,Freedom of Expression,% Healthcare Investment,% Employment Industry,% Education Expenditure,% Men Employment,% Women Employment,Population
AgriShareGDP,1.000000,-0.285154,0.438452,-0.158510,-0.073547,-0.111972,0.709065,0.265467,0.065653,-0.670833,...,0.279956,0.637313,-0.397046,-0.158882,-0.429321,-0.614526,0.376374,0.265766,-0.221063,0.034728
CreditToAgriFishForest,-0.285154,1.000000,-0.266801,0.059033,0.002433,0.101151,-0.279017,-0.047494,-0.174581,0.289674,...,-0.127330,-0.296705,0.283158,0.048481,0.291639,0.162371,-0.434429,-0.234111,0.176306,-0.035176
EmploymentRural,0.438452,-0.266801,1.000000,0.385230,0.597144,-0.175183,0.309494,-0.178467,0.104462,-0.364991,...,0.178830,0.244961,-0.371541,-0.134470,-0.187919,-0.158754,0.156863,0.452441,0.053264,0.854264
GDP,-0.158510,0.059033,0.385230,1.000000,0.636853,-0.076946,-0.189499,-0.000873,-0.100984,0.194805,...,-0.103467,-0.160186,0.111889,0.084057,0.233002,0.087314,-0.281154,0.047198,0.099591,0.472562
TotalAgri,-0.073547,0.002433,0.597144,0.636853,1.000000,-0.077201,-0.105147,-0.033252,-0.066442,0.065644,...,-0.072347,-0.012806,-0.114454,-0.124169,0.001889,0.057429,-0.124743,0.123540,0.041238,0.824867
% Soldiers,-0.111972,0.101151,-0.175183,-0.076946,-0.077201,1.000000,-0.029639,-0.148454,-0.122835,0.068762,...,0.047060,-0.255210,-0.270223,-0.333748,-0.166684,0.048024,-0.221794,-0.245135,-0.314549,-0.086256
Birth Rate,0.709065,-0.279017,0.309494,-0.189499,-0.105147,-0.029639,1.000000,0.240736,0.344360,-0.862493,...,0.565927,0.612829,-0.481966,-0.208837,-0.496292,-0.627138,0.445233,0.348488,-0.066310,-0.059658
Death Rate,0.265467,-0.047494,-0.178467,-0.000873,-0.033252,-0.148454,0.240736,1.000000,-0.213749,-0.545724,...,-0.240974,0.266246,0.053146,0.140128,-0.106418,-0.187225,-0.230842,-0.584752,-0.129713,-0.030087
Homicides,0.065653,-0.174581,0.104462,-0.100984,-0.066442,-0.122835,0.344360,-0.213749,1.000000,-0.282643,...,0.121789,0.084053,-0.222037,-0.011165,0.061876,-0.089080,0.227324,0.012054,-0.220881,-0.064456
Life Expectancy,-0.670833,0.289674,-0.364991,0.194805,0.065644,0.068762,-0.862493,-0.545724,-0.282643,1.000000,...,-0.356935,-0.630786,0.472498,0.255318,0.541090,0.551601,-0.369467,0.106269,0.267681,0.016201


# The correlation table with GDP
From this display we can start extracting valuable data in order to start deciding which of them are going to be used or not.

In [3]:
write_path = os.getcwd() + '/Output/Country'

aux = pd.DataFrame()

for file in os.listdir(write_path):



    df= pd.read_csv (write_path + '/' + file)



    cor=df[df.columns[3:]].corr()

    corcolumn=cor[['GDP']].fillna(0)
    

    aux = pd.concat((aux, corcolumn), axis = 1)

    
    


aux['average'] = aux.mean(axis=1)

aux.to_csv(write_path + '/average.csv', index = False)  

corcolumn=aux[['average']].sort_values(by = 'average',ascending = False).style.background_gradient()

display(corcolumn)

,average
GDP,0.988827
Life Expectancy,0.764903
Population,0.668125
TotalAgri,0.588413
% Healthcare Investment,0.175050
% Women Employment,0.124454
Civil Liberties,0.035531
% Education Expenditure,0.018762
Freedom of Expression,0.007373
EmploymentRural,-0.002714


In [9]:
write_path = os.getcwd() + '/Output/Region'


for file in os.listdir(write_path):

    print(file)

    df= pd.read_csv (write_path + '/' + file)


    cor=df[df.columns[3:]].corr()
     
    corcolumn=cor[['GDP']].sort_values(by = 'GDP',ascending = False).style.background_gradient()
    display(corcolumn)  

East Asia and Pacific.csv


,GDP
GDP,1.000000
TotalAgri,0.863568
Population,0.754631
EmploymentRural,0.469381
Life Expectancy,0.345394
% Employment Industry,0.272978
Death Rate,0.247324
% Healthcare Investment,0.163698
CreditToAgriFishForest,0.163409
Civil Liberties,-0.163724


Europe and Central Asia.csv


,GDP
GDP,1.000000
EmploymentRural,0.871352
TotalAgri,0.811357
Population,0.737625
CreditToAgriFishForest,0.381585
Life Expectancy,0.369034
% Healthcare Investment,0.351556
Freedom of Expression,0.239818
Civil Liberties,0.218568
% Population Growth,0.044164


Latin America and Caribbean.csv


...

,GDP
GDP,1.000000
Maternal Death Risk,0.989113
TotalAgri,0.979203
Population,0.977881
% Soldiers,0.881291
Death Rate,0.776387
% Healthcare Investment,0.727769
Infant Mortality,0.638211
% Rural Population,0.524347
Birth Rate,0.468199


South Asia.csv


,GDP
GDP,1.000000
TotalAgri,0.992562
Population,0.910447
% Employment Industry,0.406244
Civil Liberties,0.266657
Death Rate,0.159940
CreditToAgriFishForest,0.156749
Freedom of Expression,0.143317
Homicides,0.060442
Infant Mortality,0.012264


Sub-Saharan Africa.csv


,GDP
GDP,1.000000
TotalAgri,0.826406
Homicides,0.802549
Population,0.772542
% Education Expenditure,0.295875
% Employment Industry,0.277576
Freedom of Expression,0.205062
Civil Liberties,0.157222
% Healthcare Investment,0.100147
Death Rate,0.019134


In [20]:
#One dataframe per country

df= pd.read_csv (os.getcwd()+'/Output/GoldDataframe.csv')
corr_df = pd.DataFrame()
corr_df.index.names = ['Country']
aux_df = pd.DataFrame()

#List all the countries, none repeated
countries = set(df['Country'].to_list())

country_dict = {}
corr_dict = {}

for country in countries:

    #Get the DataFrame for a given country
    country_df = df.loc[df['Country'] == country]

    #Correlation matrix for that country
    country_corr_df = country_df.corr()

    #Trim it into a single row
    country_corr_df = country_corr_df.rename(columns = {'GDP': country}).drop(index = ['Year', 'GDP'])

    #Add the row to a new DataFrame with the correlations for each country
    corr_df = pd.concat([corr_df, country_corr_df[country]], axis = 1)

#Transpose the resulting DataFrame to have the desired format and show it
corr_df = corr_df.transpose()
corr_df


,AgriShareGDP,CreditToAgriFishForest,EmploymentRural,TotalAgri,% Soldiers,Birth Rate,Death Rate,Homicides,Life Expectancy,Maternal Death Risk,...,% Population Growth,% Rural Population,Civil Liberties,Freedom of Expression,% Healthcare Investment,% Employment Industry,% Education Expenditure,% Men Employment,% Women Employment,Population
Fiji,-0.424540065,NaN,NaN,0.871950751,-0.201668976,-0.910090821,0.900464142,NaN,0.952284947,-0.923800340,...,0.077927889,-0.949932007,-0.123652518,-0.349691890,-0.551848264,-0.799574276,NaN,NaN,NaN,0.907327249
Rwanda,-0.820314847,0.067505422,NaN,0.799747647,-0.842532084,-0.987402474,-0.976172616,NaN,0.975682590,-0.900568605,...,0.632970036,-0.825150703,-0.565997299,0.350529393,0.347426764,0.980806936,NaN,NaN,NaN,0.983200161
Japan,-0.491802124,NaN,NaN,0.910719042,0.393420788,-0.375864796,0.441032608,-0.443548695,0.349422639,-0.375300947,...,-0.554869510,-0.566014274,-0.034650164,-0.045267924,0.367247519,-0.466609286,-0.178459151,-0.574904684,0.016904278,0.163330116
Singapore,-0.892755062,0.771369478,NaN,0.962109244,-0.946266821,-0.876853046,0.801086384,-0.876330213,0.975188330,-0.964076955,...,-0.444153873,NaN,-0.353506614,0.866337193,0.958775118,-0.864014503,NaN,NaN,NaN,0.990639136
Liberia,-0.191644201,-0.602247614,NaN,NaN,-0.709242675,-0.954591993,-0.914437732,NaN,0.934573580,-0.899634692,...,-0.314234745,-0.956165930,0.536062436,0.495543498,-0.447416254,0.918150016,NaN,NaN,NaN,0.964365157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Turkmenistan,-0.916580803,NaN,NaN,-0.346547164,0.651003755,0.499398994,-0.945071496,NaN,0.972748957,-0.909131766,...,0.886601204,-0.949027222,0.858904013,0.688642191,-0.827309975,0.480285620,NaN,NaN,NaN,0.950748512
Angola,0.244505121,NaN,NaN,NaN,-0.810890108,-0.675961296,-0.846921071,NaN,0.837415561,-0.880650822,...,0.255430033,-0.797669842,0.610423300,0.528550548,-0.027267512,-0.377944964,NaN,NaN,NaN,0.731495203
Nauru,-0.943269772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.839543531,NaN,NaN,NaN,-0.884541305,NaN,NaN,NaN,NaN,0.766038953
Bermuda,-0.852116811,NaN,NaN,NaN,NaN,-0.809464418,0.453400571,0.787960442,0.876173085,NaN,...,-0.657904238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.673786192


In [21]:
import warnings
import matplotlib.pyplot as plt
import pandas as pd
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import statistics
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import interact, interact_manual
df= pd.read_csv (os.getcwd()+'/Output/Country/Spain.csv')

In [22]:
import ipywidgets as widgets
import plotly.express as px


indicator = widgets.SelectMultiple(
    options = corr_df.columns.tolist(),
    value = ['AgriShareGDP'],
    description='Indicator',
    disabled=False,
    layout = Layout(width='50%', height='80px')
)


def globalGrapgh(indicator):
    fig = px.choropleth(corr_df, locations = corr_df.index, locationmode='country names', 
                        color= indicator[0],projection="natural earth",
                        color_continuous_scale='Reds',
                    width=700, height=500)
    fig.update(layout_coloraxis_showscale=True)
    fig.show()
    
    
widgets.interactive(globalGrapgh, indicator = indicator) 

interactive(children=(SelectMultiple(description='Indicator', index=(0,), layout=Layout(height='80px', width='…

In [45]:
def search_indicators(threshold):   
    countryList = corr_df.index.tolist() 
    bestIndicators = {}
    transp_corr = corr_df.transpose()

    for country in countryList:
        dataAux = pd.DataFrame()
        dataAux = pd.concat([dataAux, transp_corr.loc[transp_corr[country] >= threshold, [country]]], ignore_index=False, axis=0)
        dataAux = pd.concat([dataAux, transp_corr.loc[transp_corr[country] <= -threshold, [country]]], ignore_index=False, axis=0)
        dataAux.rename(columns={ dataAux.columns[0]: "GDP Correlation" }, inplace = True)
        bestIndicators[country] = dataAux.sort_values(by=["GDP Correlation"], ascending = False)
    return bestIndicators


In [46]:
def pvalue(country):
    df= pd.read_csv (os.getcwd()+'/Output/GoldDataframe.csv')
    country_df = df.loc[df['Country'] == country]
    country_df.set_index(['Country', 'Year', 'Region'], inplace=True)

    column_pvalues = pd.DataFrame(columns=['Column', 'P-value'])

    for column in country_df.columns:
        if not country_df[column].isnull().values.any():
            column_pvalues= column_pvalues.append({'Column': column,
                                                    'P-value': stats.pearsonr(country_df[column], country_df['GDP'])[1]},
                                                    ignore_index=True) 
    column_pvalues.set_index(['Column'], inplace=True)
    return column_pvalues



In [69]:
def tableOut(Threshold, Country):
    print(pvalue(Country))
    
    df = search_indicators(Threshold)[Country]
    df  = pd.merge(df, pvalue(Country), left_index=True, right_index=True, how = 'inner')

    #df['GDP Correlation'] = df['GDP Correlation'].round(4)
    #df = np.round(df, decimals=3)
    
    left = pd.Series([0.05], index=['P-value'])
    df =df.style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:red;')\
        .set_properties(**{'background-color': '#d4dbff'}, subset=['GDP Correlation'])
    
    display(df)
    

@interact(
    Country = sorted(corr_df.index.tolist()),
    Threshold = (0, 1, 0.05))
def g(Country = 'Afghanistan', Threshold = 0.7):
    return tableOut(Threshold,Country)

    
        

interactive(children=(Dropdown(description='Country', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra',…

In [4]:
from scipy import stats
df= pd.read_csv (os.getcwd()+'/Output/Country/Switzerland.csv')

for column in df.columns[2:-1]:
    if not df[column].isnull().values.any():
        if (stats.pearsonr(df[column], df['GDP'])[0] >= 0.7 and stats.pearsonr(df[column], df['GDP'])[1] >= 0.05):
            print(column)
        if (stats.pearsonr(df[column], df['GDP'])[0] <= -0.7 and stats.pearsonr(df[column], df['GDP'])[1] >= 0.05):
            print(column)
        

In [8]:
from scipy import stats
df= pd.read_csv (os.getcwd()+'/Output/Country/Spain.csv')

for column in df.columns[2:-1]:
    print(column)
    if not df[column].isnull().values.any():
        print(stats.pearsonr(df[column], df['GDP']))

 

AgriShareGDP
(-0.9246020681684346, 2.1066936947003376e-09)
CreditToAgriFishForest
(-0.08220693589089341, 0.7231498515229101)
EmploymentRural
(-0.08370922555996547, 0.7182872885528409)
GDP
(1.0, 0.0)
TotalAgri
(0.891548865064472, 5.834448446977354e-08)
% Soldiers
(-0.8289394310900428, 3.4288731713946103e-06)
Birth Rate
(-0.07119860157494494, 0.7590842282759165)
Death Rate
(-0.466823118461258, 0.03288939980982828)
Homicides
(-0.7433876925053754, 0.00011259785360864383)
Life Expectancy
(0.6947533423591092, 0.0004738818310833547)
Maternal Death Risk
(-0.23604957926657155, 0.30294986892072606)
Infant Mortality
(-0.7344199479240557, 0.00015012647479508258)
% Population Growth
(-0.14671435316285006, 0.5256831493088687)
% Rural Population
(-0.6344726121671144, 0.0020063572013182904)
Civil Liberties
(-0.030630510474002864, 0.895140993850938)
Freedom of Expression
(-0.23027200486591226, 0.3152810108087153)
% Healthcare Investment
(0.7940862067922145, 1.7261284666949246e-05)
% Employment Industry

In [56]:
pd.options.display.float_format = '{:12.5e}'.format

In [59]:
pd.set_option('display.precision', 2)

In [67]:
pd.options.display.float_format = lambda x : '{:.0f}'.format(x) if round(x,0) == x else '{:,.10f}'.format(x)